## Вопрос 1

### Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?

In [1]:
from sklearn.metrics import f1_score, precision_score, recall_score, \
                            classification_report, confusion_matrix, accuracy_score, f1_score

In [2]:
true = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
pred = [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0]

In [3]:
confusion_matrix(true, pred)

array([[4, 1],
       [5, 5]], dtype=int64)

**MICRO**

Когда мы подсчитываем усреднение micro мы по сути берём accuracy, и не смотрим на класс. Т.е. это доля истинных срабатываний от общего числа объектов.

Данный вариант усреднения полезен, когда нет проблем accuracy (дисбаланса классов, разной цены ошибок).

In [4]:
precision = 9 / 15
precision, accuracy_score(true, pred)

(0.6, 0.6)

In [5]:
recall = 9 / 15
recall, accuracy_score(true, pred)

(0.6, 0.6)

In [6]:
2 * (precision * recall) / (precision + recall)

0.6

In [7]:
f1_score(true, pred, average='micro')

0.6

**MACRO**

В данном случае мы подсчитываем среднее для метрик подсчитанных для каждого класса.

 Данный вариант усреднения учитывает ошибки в рамках каждого класса.

In [8]:
precision_0 = 4 / (4 + 5)
precision_1 = 5 / (5 + 1)
macro_pr = (precision_1 + precision_0) / 2
macro_pr

0.6388888888888888

In [9]:
precision_score(true, pred, average='macro')

0.6388888888888888

In [10]:
recall_1 = 5 / (5 + 5)
recall_0 = 4 / (4 + 1)
macro_rec = (recall_1 + recall_0) / 2
macro_rec

0.65

In [11]:
recall_score(true, pred, average='macro')

0.65

In [12]:
# 1 способ
f_score_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0)
f_score_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1)
f_score = (f_score_0 + f_score_1) / 2
f_score_0, f_score_1, f_score

(0.5714285714285714, 0.625, 0.5982142857142857)

In [13]:
f1_score(true, pred, average='macro')

0.5982142857142857

**WEIGHTED**

Учитывает количество объектов каждого класса, и чем больше объектов какого-либо класса, то и правильность работы на этом классе оценивается выше (имеет больший вес).

Полезно если нам важно чтобы модель работала хорошо на объектах мажорного класса (которых больше), и нам не критично как она работает на объектах минорных классов.

In [14]:
zero = true.count(0)
one = true.count(1)
size = len(true)
zero / size * precision_0 + one / size * precision_1

0.7037037037037037

In [15]:
precision_score(true, pred, average='weighted')

0.7037037037037038

In [16]:
zero / size * recall_0 + one / size * recall_1

0.6

In [17]:
recall_score(true, pred, average='weighted')

0.6

In [18]:
f_score_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0)
f_score_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1)
zero / size * f_score_0  + one / size * f_score_1

0.6071428571428571

In [19]:
f1_score(true, pred, average='weighted')

0.6071428571428572

## Вопрос 2

### В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

#### XGBoost (eXtreme Gradient Boosting)

***1. Производит обучение на полной выборке (используются значения по всем строкам, и всем признакам).***

***2. При обучении использует критерий информативности "Прирост" на базе критерия "Похожесть".***

<center><h4><i>Похожесть</i></h4></center><br>
$$Similarity = \frac{(\sum{Resideal_i})^2}{\sum{[Prev Proba_i * (1 - Prev Proba_i)]} + \lambda}$$

<center><h4><i>Прирост</i></h4></center><br>
$$Gain = Left_{Similarity} + Right_{Similarity} - Root_{Similarity}$$

***3. Построение дерева - как в классическом бустинге, по уровням.***

#### LightGBM

***1. Производит обучение на ограниченной выборке.***

*Gradient-based One-Side Sampling (GOSS)*

GOSS сохраняет наблюдения с большим градиентом (т.е. где больше ошибка) и случайно сэмплирует выборку из наблюдений с маленькими градиентами.
Итого, выбирается a ∗ 100% наблюдений из топа с большими градиентами.
Рандомно b ∗ 100% наблюдений из остальной.

*Exclusive Feature Bundling (EFB)*

Разряжённые данные значат, что многие признаки никогда ненулевые вместе. Это позволяет совместить ("bundle") эти фичи в одну без потери информации.<br>
Пример, у фичи А диапазон значений (0, 10) и у B - (0, 20). Добавляется смещение 10 к значениям фичи B, теперь их диапазон (10, 30). После, можно соединять A и B и использовать "feature bundle" с диапазоном (0, 30), чтобы заменить оригинальные A и B.

***2. Построение дерева - по листьям. На очередной итерации выбирается лист где больше ошибка***

<img src="images/lgbm_tree.png">

#### CatBoost

***1. Построение дерева - симметричное***

<img src="images/catboost.png">

***2. Реализована собственная логика работы с категориальными признаками***

***3. Содержит логику направленную на предотвращение переобучения***

   - Обучает несколько моделей за одну итерацию, отклонение вычисляется по наблюдениям на тех моделях, которые не видели его.
   - Высокий уровень рандома (перемешивание выборки и много случайности в начале построения дерева)